<a href="https://colab.research.google.com/github/JyzMinaBF/Retirement_Calculation/blob/main/Pension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

life_male = pd.read_csv("https://raw.githubusercontent.com/JyzMinaBF/Miscellaneous/main/life_male.csv")
life_female = pd.read_csv("https://raw.githubusercontent.com/JyzMinaBF/Miscellaneous/main/life_female.csv")

77.64


In [ ]:
import numpy as np

def month_to_pay_change(year_in_pension, retire_age):
  if year_in_pension <= 15:
    month_to_pay = year_in_pension
  elif year_in_pension > 15:
    month_to_pay = np.min([2 * (year_in_pension - 15) + 15, 45])

  if month_to_pay == 45 and retire_age > 60:
    month_to_pay = np.min([month_to_pay + retire_age - 60, 50])

  return month_to_pay

def once_for_all_bef(fee_month_36, year_in_pension, retire_age):
  month_to_pay = month_to_pay_change(year_in_pension, retire_age)
  return fee_month_36 * month_to_pay

def once_for_all_aft(fee_month_60, year_in_pension, retire_age):
  month_to_pay = month_to_pay_change(year_in_pension, retire_age)
  return fee_month_60 * month_to_pay

def receive_cont(fee_month_60, year_in_pension, diff_of_age):
  st1 = fee_month_60 * year_in_pension * 0.00775 + 3000
  st2 = fee_month_60 * year_in_pension * 0.0155
  return np.max([st1, st2]) * (1 + 0.04 * diff_of_age)

def change(birth_year):
    if birth_year <= 46:
      return 60
    if birth_year in (47, 48, 49, 50):
      return birth_year + 14
    else:
      return 65

def discount(cf, r, year):
  n = int(round(12 * year, 0))
  pv = sum([round(cf / round((1 + r)**i, 20), 20) for i in range(1, n+1)])
  return pv

def throw_back(cf, r, pv_threshold):
  n = 0       # Initialize time period
  pv = 0      # Initialize present value

  while pv < pv_threshold:
    pv += cf / round((1 + r)**n, 20)
    n += 1

  return round(n/12, 3)

def range_calculated(year_from_105):
  period = year_from_105//9
  if year_from_105 <= 7:
    smallest = 26400
  else:
    smallest = 26400* (1.013 ** (year_from_105-7))
  biggest = 43900 * (1.045 ** period)
  diff = (biggest - smallest) / 12
  return [smallest + diff * i for i in range(13)]

def level_calculated(fee, birth_year, retire_age):
  ranges = range_calculated(birth_year + retire_age - 105)
  if fee <= ranges[0]:
    return ranges[0]
  elif fee > ranges[12]:
    return ranges[12]
  else:
    for i in range(len(ranges)-1):
      if fee > ranges[i] and fee <= ranges[i+1]:
        return ranges[i+1]

def condition_loop(fee_month_60, birth_year, retire_age, year_in_pension, diff_of_age, expected_inflation_rate, expected_life):
  diff_to_base = diff_of_age + 5
  base_age = retire_age - diff_to_base
  base_year_in_pension = year_in_pension - diff_to_base
  results = {}
  print("\n此外，以下為你在不同年齡退休下，如活至預期壽命時，折現至你現今預期退休年齡的金額：")
  for i in range(11):
    present_age = base_age + i
    present_year_in_pension = base_year_in_pension + i
    present_fee = level_calculated(fee_month_60, birth_year, present_age)
    month_money = receive_cont(present_fee, present_year_in_pension, i - 5)
    total_money = discount(month_money, expected_inflation_rate/12, expected_life - present_age) / ((1 + expected_inflation_rate) ** (i - diff_to_base))
    results[total_money] = present_age
    print("年齡：", present_age, " 可收：", "%.2f"%total_money)
  return "綜上所述，最佳的退休年齡是：" + str(results[max(results.keys())])



def pension(birth_year, year_in_pension, retire_age, sex, year_in_dang_job, fee_month_36, fee_month_60,
            longest_in_one_company, nation_year, expected_inflation_rate, expected_life, 民國九十八年以前曾投保):
  try:
    birth_year = int(birth_year)
    year_in_pension = float(year_in_pension)
    retire_age = int(retire_age)
    year_in_dang_job = int(year_in_dang_job)
    fee_month_36 = int(fee_month_36)
    fee_month_60 = int(fee_month_60)
    longest_in_one_company = int(longest_in_one_company)
    nation_year = int(nation_year)
    expected_inflation_rate = float(expected_inflation_rate)
    if expected_life == "0":
      if sex == "男":
        expected_life = retire_age + life_male.iloc[retire_age, 1]
      else:
        expected_life = retire_age + life_female[retire_age, 1]
    else:
      expected_life = int(expected_life)
  except:
    return "請輸入數字。"

  fee_month_36_changed = level_calculated(fee_month_36, birth_year, retire_age)
  fee_month_60_changed = level_calculated(fee_month_60, birth_year, retire_age)

  cond1, cond2, cond3 = False, False, False

  if 民國九十八年以前曾投保 == True:
    if sex == "女" and year_in_pension >= 1 and retire_age >= 55:
      cond1 = True
    elif sex == "男" and year_in_pension >= 1 and retire_age >= 60:
      cond1 = True
    elif year_in_pension >= 15 and retire_age >= 55:
      cond1 = True
    elif longest_in_one_company >= 25:
      cond1 = True
    elif year_in_pension >= 25 and retire_age >= 50:
      cond1 = True
    elif year_in_dang_job >= 5 and retire_age >= 55:
      cond1 = True

  limit_age = change(birth_year)
  diff_of_age = retire_age - limit_age

  if diff_of_age >= -5:
    if year_in_pension < 15:
      cond2 = True
    elif year_in_pension >= 15:
      cond3 = True

  if year_in_dang_job >= 15 and retire_age >= 55:
    cond3 = True
  elif (nation_year + year_in_pension) >= 15 and retire_age >= 65:
    cond3 = True

  print("可以選擇的請領方式：\n")
  if cond1:
    money_1 = once_for_all_bef(fee_month_36_changed, year_in_pension, retire_age)
    print("一次請領老年給付：可以")
    print("此方法可請領金額為：", "%.2f"%money_1, "\n")
  else:
    money_1 = 0
    print("一次請領老年給付：無法", "\n")

  if cond2:
    money_2 = once_for_all_aft(fee_month_60_changed, year_in_pension, retire_age)
    print("老年一次金給付：可以")
    print("此方法可請領金額為：", "%.2f"%money_2, "\n")
  else:
    money_2 = 0
    print("老年一次金給付：無法", "\n")

  if cond3:
    money_3 = receive_cont(fee_month_60_changed, year_in_pension, diff_of_age)
    print("老年年金給付：可以")
    print("每月可領取金額為：", "%.2f"%money_3)
    print("領取五年折現將為：", "%.2f"%discount(money_3, expected_inflation_rate/12, 5))
    print("領取十年折現將為：", "%.2f"%discount(money_3, expected_inflation_rate/12, 10))
    print("領取十五年折現將為：", "%.2f"%discount(money_3, expected_inflation_rate/12, 15))
    print("領取二十年折現將為：", "%.2f"%discount(money_3, expected_inflation_rate/12, 20))
    print("領取至您預期壽命的「", "%.2f"%expected_life, "」歲折現將為：", discount(money_3, expected_inflation_rate/12, expected_life - retire_age))
    if cond3 and (cond1 or cond2):
      print("此方法相較一次領之方法如再存活至:", retire_age + throw_back(money_3, expected_inflation_rate/12, np.max([money_1, money_2])), "歲，將可領得更多")
    print(condition_loop(fee_month_60, birth_year, retire_age, year_in_pension, diff_of_age, expected_inflation_rate, expected_life))
  else:
    money_3 = 0
    print("老年年金給付：無法")


In [ ]:
from ipywidgets import interact_manual
interact_manual(
    pension,
    birth_year = "請輸入出生年（民國）",
    year_in_pension = "請輸入參加保險年資",
    retire_age = "請輸入預計退休年齡",
    sex = ["男", "女"],
    year_in_dang_job = "請輸入擔任危險工作之年資（如無請輸入0）",
    fee_month_36 = "請輸入最高薪資之三十六個月之平均",
    fee_month_60 = "請輸入最高薪資六十個月之平均",
    longest_in_one_company = "請輸入在一間公司所待最長年資（如未於民國98年前投保可填0）",
    nation_year = "國民年金繳納年資",
    expected_inflation_rate = "預計之每年通貨膨脹率",
    expected_life = "預期壽命，如預計使用預設的預期壽命，請輸入0。",
    民國九十八年以前曾投保 = True
);

interactive(children=(Text(value='請輸入出生年（民國）', description='birth_year'), Text(value='請輸入參加保險年資', description=…